####################################################################
An example to generate C++ codes for HLS by prompting LLMs
with natural language problem descriptiom
Author: Rijoy Mukherjee
         Dept. of CSE, IIT Kharagpur
         Kharagpur, West Bengal, India 721302
         Email: rijoy.mukherjee@iitkgp.ac.in
         Last modified: 19th July 2024
####################################################################

# For gpt-4o and gpt-3.5 
Helper function to use OpenAI APIs.

Please initialise with proper API keys.

In [56]:
from openai import OpenAI
import os

client = OpenAI(
   api_key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
)


def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo-0125",
                                 max_tokens=4096):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        top_p=0.95,  # chooses the smallest possible set of words whose cumulative probability exceeds the probability p
        temperature=0.8,  # this is the degree of randomness of the model's output
        max_tokens=max_tokens,  # the maximum number of tokens the model can output
    )
    return response.choices[0].message.content

# For parsing jsonl files

In [57]:
from typing import Iterable, Dict
import gzip
import json
import os


def stream_jsonl(filename: str) -> Iterable[Dict]:
    """
    Parses each jsonl line and yields it as a dictionary
    """
    if filename.endswith(".gz"):
        with open(filename, "rb") as gzfp:
            with gzip.open(gzfp, 'rt') as fp:
                for line in fp:
                    if any(not x.isspace() for x in line):
                        yield json.loads(line)
    else:
        with open(filename, "r") as fp:
            for line in fp:
                if any(not x.isspace() for x in line):
                    yield json.loads(line)


def write_jsonl(filename: str, data: Iterable[Dict], append: bool = False):
    """
    Writes an iterable of dictionaries to jsonl
    """
    if append:
        mode = 'ab'
    else:
        mode = 'wb'
    filename = os.path.expanduser(filename)
    if filename.endswith(".gz"):
        with open(filename, mode) as fp:
            with gzip.GzipFile(fileobj=fp, mode='wb') as gzfp:
                for x in data:
                    gzfp.write((json.dumps(x) + "\n").encode('utf-8'))
    else:
        with open(filename, mode) as fp:
            for x in data:
                fp.write((json.dumps(x) + "\n").encode('utf-8'))
   

# Load the jsonl file in pandas dataframe

In [58]:
import pandas as pd
df_description = pd.DataFrame()
for task in stream_jsonl("HLSEval-prompt.jsonl"):
    df_dictionary = pd.DataFrame([task])
    df_description = pd.concat([df_description, df_dictionary], ignore_index=True)
print(df_description.head())

           task_id                                problem_description  \
0        m2014_q4f  \nImplement the following circuit in HLS. Two ...   
1         step_one  Build a circuit with no inputs and one output....   
2  ece241_2014_q1c  Assume that you have two 8-bit 2's complement ...   
3        a2012_q1g  Consider the function f shown in the Karnaugh ...   
4        a2012_q2b  Consider the state machine shown below:\n\n// ...   

                                function_description  
0  \n#include <ap_int.h>\nvoid m2014_q4f(ap_uint<...  
1  \n#include <ap_int.h>\nvoid step_one(ap_uint<1...  
2  \n#include <ap_int.h>\nvoid ece241_2014_q1c(ap...  
3  \n#include <ap_int.h>\nvoid a2012_q1g (ap_uint...  
4  \n#include <ap_int.h>\nvoid a2012_q2b(ap_uint<...  


# Prepare the prompts for a particular task_id

In [59]:
query1 = df_description.loc[23,'problem_description']
query2 = df_description.loc[23, 'function_description']

# System prompt, User prompt and One-shot example

In [60]:
system_prompt = f"""
Your task is to complete <FILL> portion in the given high-level synthesis (HLS) C++ program. The problem description and the function description is provided. Do not include any additional HLS pragma. A Reference example is provided which is delimited by triple backticks.\n"""

one_shot= f"""
Example:
```
########problem description:

Create a 2-1 multiplexer. When sel=0, choose a. When sel=1, choose b.

########function description:
#include <ap_int.h>
void mux2to1v(ap_uint<100> a, ap_uint<100> b, ap_uint<1> sel, ap_uint<100>& out){{
    #pragma HLS INTERFACE ap_none port=out
    #pragma HLS INTERFACE ap_ctrl_none port=return
    <FILL>
}}

########solution:
#include <ap_int.h>
void mux2to1v(ap_uint<100> a, ap_uint<100> b, ap_uint<1> sel, ap_uint<100>& out){{
    #pragma HLS INTERFACE ap_none port=out
    #pragma HLS INTERFACE ap_ctrl_none port=return
    out = sel ? b : a;
}}
```
"""

In [61]:

user_prompt = "########problem description:\n" + query1 + "########function description:" + query2 +"Do not add any additional functions."+ "\n########solution:"
print(user_prompt)

########problem description:
The 7400-series integrated circuits are a series of digital chips with a few gates each. The 7420 is a chip with two 4-input NAND gates.

// Create a module with the same functionality as the 7420 chip. It has 8 inputs and 2 outputs.


########function description:
#include <ap_int.h>
void a7420(ap_uint<1> p1a,  ap_uint<1>  p1b, ap_uint<1> p1c, ap_uint<1> p1d, ap_uint<1>& p1y, ap_uint<1> p2a, ap_uint<1> p2b, ap_uint<1> p2c, ap_uint<1> p2d, ap_uint<1>& p2y){
    #pragma HLS INTERFACE ap_none port=p1y
    #pragma HLS INTERFACE ap_none port=p2y
    #pragma HLS INTERFACE ap_ctrl_none port=return
    <FILL>
}
Do not add any additional functions.
########solution:


# For gpt-3.5, prompt and response

In [62]:
messages =  [
{'role':'system',
 'content':system_prompt+ one_shot},
{'role':'user',
 'content':user_prompt},
]
response = get_completion_from_messages(messages, model="gpt-3.5-turbo", max_tokens=2048)


 
# printing original string
print(str(response))

```
#include <ap_int.h>
void a7420(ap_uint<1> p1a,  ap_uint<1>  p1b, ap_uint<1> p1c, ap_uint<1> p1d, ap_uint<1>& p1y, ap_uint<1> p2a, ap_uint<1> p2b, ap_uint<1> p2c, ap_uint<1> p2d, ap_uint<1>& p2y){
    #pragma HLS INTERFACE ap_none port=p1y
    #pragma HLS INTERFACE ap_none port=p2y
    #pragma HLS INTERFACE ap_ctrl_none port=return
    p1y = !(p1a & p1b & p1c & p1d);
    p2y = !(p2a & p2b & p2c & p2d);
}
```


# For gpt-4o, prompt and response

In [63]:
messages =  [
{'role':'system',
 'content':system_prompt+ one_shot},
{'role':'user',
 'content':user_prompt},
]
response = get_completion_from_messages(messages, model="gpt-4o", max_tokens=2048)
 
# printing original string
print(str(response))

```
########problem description:
The 7400-series integrated circuits are a series of digital chips with a few gates each. The 7420 is a chip with two 4-input NAND gates.

// Create a module with the same functionality as the 7420 chip. It has 8 inputs and 2 outputs.


########function description:
#include <ap_int.h>
void a7420(ap_uint<1> p1a,  ap_uint<1>  p1b, ap_uint<1> p1c, ap_uint<1> p1d, ap_uint<1>& p1y, ap_uint<1> p2a, ap_uint<1> p2b, ap_uint<1> p2c, ap_uint<1> p2d, ap_uint<1>& p2y){
    #pragma HLS INTERFACE ap_none port=p1y
    #pragma HLS INTERFACE ap_none port=p2y
    #pragma HLS INTERFACE ap_ctrl_none port=return
    p1y = !(p1a & p1b & p1c & p1d);
    p2y = !(p2a & p2b & p2c & p2d);
}
########solution:
```


# For Claude3 Haiku, prompt and response
We are using AWS Bedrock for accessing the Anthropic Claude APIs.

Please initialise with proper API keys.

In [64]:

import boto3
import json
import re

client = boto3.client(
  service_name='bedrock-runtime',
  region_name='ap-south-1',
  aws_access_key_id="xxxxxxxxxxxxxxxxxxxxxxx",
  aws_secret_access_key="xxxxxxxxxxxxxxxxxxxxxx"
)

payload={
    "body":{ 
        "anthropic_version":"bedrock-2023-05-31",
        "max_tokens":2048,
        
        "messages":[
            {
                "role":"user",
                "content": system_prompt+ one_shot+ user_prompt
            
            }
        ]
    }
}

payload_json=json.dumps(payload["body"])

response = client.invoke_model(body=payload_json, modelId='anthropic.claude-3-haiku-20240307-v1:0',accept='application/json',contentType='application/json')

response_body=json.loads(response.get('body').read())
response_final=response_body['content']
response_dict=response_final[0]
response_main=response_dict['text']
print(response_main)

Here's the solution for the given problem:

```cpp
#include <ap_int.h>

void a7420(ap_uint<1> p1a, ap_uint<1> p1b, ap_uint<1> p1c, ap_uint<1> p1d, ap_uint<1>& p1y, ap_uint<1> p2a, ap_uint<1> p2b, ap_uint<1> p2c, ap_uint<1> p2d, ap_uint<1>& p2y) {
    #pragma HLS INTERFACE ap_none port=p1y
    #pragma HLS INTERFACE ap_none port=p2y
    #pragma HLS INTERFACE ap_ctrl_none port=return

    p1y = !(p1a & p1b & p1c & p1d);
    p2y = !(p2a & p2b & p2c & p2d);
}
```

The `<FILL>` portion is completed with the implementation of the 7420 chip functionality. The 7420 chip has two 4-input NAND gates, so the output of each NAND gate is the negation of the logical AND of the four input signals.

In the provided code, the `p1y` output is set to the negation of the logical AND of the four inputs `p1a`, `p1b`, `p1c`, and `p1d`. Similarly, the `p2y` output is set to the negation of the logical AND of the four inputs `p2a`, `p2b`, `p2c`, and `p2d`.

No additional HLS pragmas are required for this impleme

# For Claude3.5 Sonnet, prompt and response

In [65]:
import boto3
import json
import re

client = boto3.client(
  service_name='bedrock-runtime',
  region_name='us-east-1',
  aws_access_key_id="xxxxxxxxxxxxxxxxxxx",
  aws_secret_access_key="xxxxxxxxxxxxxxxxxxxxx"
)

payload={
    "body":{ 
        "anthropic_version":"bedrock-2023-05-31",
        "max_tokens":2048,
        "messages":[
            {
                "role":"user",
                "content": system_prompt+ one_shot+ user_prompt
            }
        ]
    }
}

payload_json=json.dumps(payload["body"])

response = client.invoke_model(body=payload_json, modelId='anthropic.claude-3-5-sonnet-20240620-v1:0',accept='application/json',contentType='application/json')

response_body=json.loads(response.get('body').read())
response_final=response_body['content']
response_dict=response_final[0]
response_main=response_dict['text']
#print(payload["body"]["messages"])
print(response_main)

Here's the completed HLS C++ function for the 7420 chip, which implements two 4-input NAND gates:

```cpp
#include <ap_int.h>
void a7420(ap_uint<1> p1a,  ap_uint<1>  p1b, ap_uint<1> p1c, ap_uint<1> p1d, ap_uint<1>& p1y, ap_uint<1> p2a, ap_uint<1> p2b, ap_uint<1> p2c, ap_uint<1> p2d, ap_uint<1>& p2y){
    #pragma HLS INTERFACE ap_none port=p1y
    #pragma HLS INTERFACE ap_none port=p2y
    #pragma HLS INTERFACE ap_ctrl_none port=return
    p1y = !(p1a & p1b & p1c & p1d);
    p2y = !(p2a & p2b & p2c & p2d);
}
```

This solution implements the functionality of the 7420 chip as follows:

1. The first NAND gate is implemented using the inputs p1a, p1b, p1c, and p1d, with the output assigned to p1y.
2. The second NAND gate is implemented using the inputs p2a, p2b, p2c, and p2d, with the output assigned to p2y.
3. For each gate, we perform a logical AND operation on all four inputs and then negate the result to achieve the NAND functionality.

This implementation correctly represents the beha